# 자연어 데이터셋 전처리 
- Korpora 패키지 활용
- 데이터셋 : 한국어 혐오 데이터셋 korean_hate_speech

In [30]:
# !pip install Korpora

In [31]:
import Korpora

In [32]:
Korpora.__version__

'0.2.0'

# 1. 데이터 준비

In [33]:
from Korpora import Korpora

# 데이터셋 파일명
data_file = "korean_hate_speech"

# 다운로드 fetch
corpus = Korpora.fetch(data_file)

[Korpora] Corpus `korean hate speech` is already installed at C:\Users\kdp\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_1.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\kdp\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_2.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\kdp\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_3.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\kdp\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_4.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\kdp\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_5.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\kdp\Korpora\korean_hate_speech\news_title\unlabeled_comments.news_title_1.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\kdp\Korpora\korean_hate_speech\news_title\unlabeled_comments.news_title_2.txt
[Korpora] Corpus 

In [34]:
# 데이터 로드
corpus = Korpora.load(data_file)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Authors :
        - Jihyung Moon* (inmoonlight@github)
        - Won Ik Cho* (warnikchow@github)
        - Junbum Lee (beomi@github)
        * equal contribution
    Repository : https://github.com/kocohub/korean-hate-speech
    References :
        - Moon, J., Cho, W. I., & Lee, J. (2020). BEEP! Korean Corpus of Online News
          Comments for Toxic Speech Detection. arXiv preprint arXiv:2005.12503.

    We provide the first human-annotated Korean corpus for toxic speech detection and the large unlabeled corpus.
    The data is comments from the Korean entertainment news aggregation platform.

    # License
    Creative Commons Attribution-ShareAlike 4.0 International License.
    Visit here for detail : https://creativec

In [35]:
corpus.train[0] # 데이터 확인

KoreanHateSpeechLabeledExample(text='(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....', title='"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"', gender_bias='False', bias='others', hate='hate')

In [36]:
# 데이터 확인
hateDS = corpus.train

# 1개 데이터 정보 추출
print(hateDS[0].text)

(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....


In [37]:
hateDS[0].__dict__ # 볼 수 있는 속성들

{'text': '(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....',
 'title': '"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"',
 'gender_bias': 'False',
 'bias': 'others',
 'hate': 'hate'}

In [38]:
hateDS = corpus.train # train
hate_valDS = corpus.dev # val
hate_testDS = corpus.test # test

unlabeledDS = corpus.unlabeled # 라벨이 없음 -> 걍 predict할 때 사용하면 됨

In [39]:
unlabeledDS[0].text

'"[단독] 지드래곤♥이주연, 제주도 데이트…2018년 1호 커플 탄생"'

# 2. 데이터 전처리 
- 주제에 따른 데이터 피쳐 & 라벨 선택 => 뉴스댓글 + 차별 종류
- 형태소 분석기 결정

In [40]:
# 피쳐 데이터 추출
textData = corpus.get_all_texts()

In [41]:
# 데이터 파일 저장
with open("corpus.txt", "w", encoding="utf-8") as f:
    for text in textData:
        f.write(text + "\n")

In [45]:
# SentencePieceTrainer -> 토크나이저 모델 학습 진행
from sentencepiece import SentencePieceTrainer
'''
매개변수
input : 말뭉치 텍스트 파일의 경로
model_prefix : 모델 파일 이름
vocab_size : 어휘 사전 크기
character_coverage : 말뭉치 내에 존재하는 글자 중 토크나이저가 다룰 수 있는 글자 비율
...
'''
SentencePieceTrainer.train(
    "--input=corpus.txt \
    --model_prefix=text_bpe \
    --vocab_size=8000 \
    --model_type=bpe"
)

# 결과로 text_bpe.model(학습이 된 모델), text_bpe.vocab(단어 사전) 생성 

In [52]:
# 위에서 학습된 모델과 어휘 사전 파일을 활용해 바이트 페어 인코딩 수행
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()
tokenizer.Load("text_bpe.model") # 모델 로드

sentence = "안녕하세요, 토크나이저가 잘 학습되었군요!"
sentences = ["이렇게 입력값을 리스트로 받아서", "쉽게 토크나이저를 사용할 수 있답니다"]

tokenized_sentence = tokenizer.encode_as_pieces(sentence)
tokenized_sentences = tokenizer.encode_as_pieces(sentences)

In [53]:
print("단일 문장 토큰화 :", tokenized_sentence)
print("여러 문장 토큰화 :", tokenized_sentences)

encoded_sentence = tokenizer.encode_as_ids(sentence)
encoded_sentences = tokenizer.encode_as_ids(sentences)
print("단일 문장 정수 인코딩 :", encoded_sentence)
print("여러 문장 정수 인코딩 :", encoded_sentences)

decode_ids = tokenizer.decode_ids(encoded_sentences)
decode_pieces = tokenizer.decode_pieces(encoded_sentences)
print("정수 인코딩에서 문장 변환 :", decode_ids)
print("하위 단어 토큰에서 문장 변환 :", decode_pieces)

단일 문장 토큰화 : ['▁안녕', '하세요', ',', '▁토크', '나이', '저', '가', '▁잘', '▁학', '습', '되', '었', '군', '요', '!']
여러 문장 토큰화 : [['▁이렇게', '▁입', '력', '값', '을', '▁리', '스트', '로', '▁받아', '서'], ['▁', '쉽', '게', '▁토크', '나이', '저', '를', '▁사용', '할', '▁수', '▁있', '답', '니다']]
단일 문장 정수 인코딩 : [4598, 1073, 6681, 3614, 2850, 6901, 6711, 290, 816, 7105, 7102, 6929, 7149, 6812, 6799]
여러 문장 정수 인코딩 : [[1646, 116, 7000, 7541, 6924, 533, 314, 6709, 1561, 6726], [6677, 7525, 6806, 3614, 2850, 6901, 7124, 3689, 6886, 95, 146, 7195, 264]]
정수 인코딩에서 문장 변환 : ['이렇게 입력값을 리스트로 받아서', '쉽게 토크나이저를 사용할 수 있답니다']
하위 단어 토큰에서 문장 변환 : ['이렇게 입력값을 리스트로 받아서', '쉽게 토크나이저를 사용할 수 있답니다']
